# This Program takes a reference dictionary and codes your patient list of medications based on the (code) (indication) and (drug_name) variables.


In [1]:
#dic = input("Dictionary file location: ")
#wrong = input("Project file location: ")
#end_result = input("Output file name: ")
dic='/home/michael/Downloads/felsndc.csv'
wrong= "/home/michael/Downloads/sampleuncoded.csv"

In [2]:
#importing numpy, pandas, cvs, and pyenchant libraries to work on the code
!pip install pyenchant
import pandas as pd
import numpy as np
import csv
import enchant
#uncoded = pd.read_csv(urllib.quote("'" + wrong + "'"))
#dictionary = pd.read_csv(urllib.quote("'" + dic + "'"))
uncoded = pd.read_csv(wrong)
dictionary = pd.read_csv(dic)

In [3]:
uncoded.head() #This takes a look at the data in uncoded

,meds,fauxid
0,BYSTOLIC TABLETS,1.0
1,"lisinopril,ACTIFED",2.0
2,HALFPRIN,3.0
3,"TOPROL EXTENDED RELEASE TABLET,LYRICA CAPSULE,...",4.0
4,DLITIAZEM HCL CAPSULE,5.0


In [4]:
dictionary.head() #This takes a look at the data in dictionary

,listorder,Maj_Therap_Clss_Cde,Code,Drug_Name_Code,drug_name,indication
0,1,100,100,165,BIER BLOCK,NaN
1,2,100,100,1983,ANESTHETIC,NaN
2,3,100,100,2396,VARICELLA VIRUS VACCINE,NaN
3,4,100,117,130,A-CAINE,"ANESTHETICS, LOCAL"
4,5,100,117,1154,XYLOCAINE/MARCAINE,"ANESTHETICS, LOCAL"


In [5]:
print(dictionary['drug_name'].values)

['BIER BLOCK' 'ANESTHETIC' 'VARICELLA VIRUS VACCINE' ..., 'RUBBER CEMENT'
 'HEROIN' 'PCP']


In [6]:
#Here we are working on our data set

In [7]:
#nospace = pd.concat([uncoded['fauxid'],uncoded['meds'].str.replace(' ', '_', case=False)], axis=1) 


In [8]:
comma_only = pd.concat([uncoded['fauxid'],uncoded['meds'].str.replace(';|/', ',', case=False)], axis=1) 
#Here we replace semi colons and slashes with commas

In [9]:
split_meds = pd.concat([uncoded['fauxid'],comma_only['meds'].str.split(',', expand=True)], axis=1)
#Now we split by commas

In [10]:
split_meds.head()
#This is the reformatted version of our original meds variable in split_meds

,fauxid,0,1,2,3,4,5,6,7,8,9
0,1.0,BYSTOLIC TABLETS,None,None,None,None,None,None,None,None,None
1,2.0,lisinopril,ACTIFED,None,None,None,None,None,None,None,None
2,3.0,HALFPRIN,None,None,None,None,None,None,None,None,None
3,4.0,TOPROL EXTENDED RELEASE TABLET,LYRICA CAPSULE,HALFPRIN,None,None,None,None,None,None,None
4,5.0,DLITIAZEM HCL CAPSULE,None,None,None,None,None,None,None,None,None


In [11]:
nmeds=(((comma_only['meds'].str.split(',', expand=True))).shape[1])
#gives the number of new columns given from splitting meds by commas

In [12]:
for i in range(nmeds):
    print(i)
#example of listing all columns in nmeds

0
1
2
3
4
5
6
7
8
9


In [13]:
print(nmeds)

10


In [14]:
#Now we can develop our dictionary

In [15]:
dictionary['drug_name'].head()

0                 BIER BLOCK
1                 ANESTHETIC
2    VARICELLA VIRUS VACCINE
3                    A-CAINE
4         XYLOCAINE/MARCAINE
Name: drug_name, dtype: object

In [16]:
dog=str.split(dictionary['drug_name'].to_string(index=False)) #dataframe to string w/ one row per word
weirddog=pd.DataFrame([dog]) #Tulap to dataframe by column
gooddog=pd.DataFrame.transpose(weirddog) #transposing
#print(gooddog.to_string(index=False)).values
drug_name=(gooddog.values)
print(weirddog.head())
print(type(weirddog))
print(gooddog.head())
print(drug_name)

  0      1           2          3      4        5        6      \
0  BIER  BLOCK  ANESTHETIC  VARICELLA  VIRUS  VACCINE  A-CAINE   

                7          8           9      ...       17061  17062  \
0  XYLOCAINE/MARCAINE  RAVOCAINE  AMERICAINE  ...   (ILLICIT)  SPEED   

                   17063     17064   17065    17066   17067   17068   17069  \
0  GAMMA-HYDROXYBUTYRATE  GASOLINE  POLISH  REMOVER  RUBBER  CEMENT  HEROIN   

  17070  
0   PCP  

[1 rows x 17071 columns]
            0
0        BIER
1       BLOCK
2  ANESTHETIC
3   VARICELLA
4       VIRUS
[['BIER']
 ['BLOCK']
 ['ANESTHETIC']
 ..., 
 ['CEMENT']
 ['HEROIN']
 ['PCP']]


In [17]:
np.savetxt('drug_name.txt', drug_name, fmt ='%s') 
#Save the variable into a .txt file

In [18]:
pwl = enchant.request_pwl_dict('drug_name.txt') 
#Import the .txt into enchant as a custom made dictionary

In [19]:
pwl.check('ANESTHETIC') 
#The spellcheck is able to recognize exact matches

True

In [20]:
pwl.check('Anesthetic') 
#Spellcheck is case sensitive. Dictionary and Variables being checked will need to be put into upercase

False

In [21]:
pwl.suggest('Anesthetic')
#This functions outputs a suggestion code_string = (pd.Series.to_string(dictionary.Code))
#For the next section of code we need to turn our variable into a string
#This line takes the code variable of the dictionary data set 
#and converts it into a string function

['ANESTHETIC']

In [22]:
#Looped SpellChecker
i = 0
main_df = pd.DataFrame()
print((split_meds).shape)
while i < nmeds:
    from enchant.checker import SpellChecker
    chkr = SpellChecker(pwl)
    chkr.set_text(pd.Series.to_string(split_meds[i]))
    for err in chkr:
        if pwl.suggest(err.word) != []:
            df=(split_meds[i].str.replace(err.word, pwl.suggest(err.word)[0]))
        elif pwl.suggest(err.word) == []:
            df=(split_meds[i].str.replace(err.word, err.word))
        #df=(split_meds[i].str.replace(err.word, pwl.suggest(err.word)[0]))
        #print(df)
        if main_df.empty:
            main_df=pd.Series.to_frame(df)
        else:
            main_df=pd.concat([main_df, df], axis=1)
    i=i+1
print(main_df.shape)

#pd.DataFrame.to_csv(main_df.head(10))


(1211, 11)
(1211, 11086)


In [23]:
print(split_meds[0].to_string(index=False))

BYSTOLIC TABLETS
                                      lisinopril
                                        HALFPRIN
                  TOPROL EXTENDED RELEASE TABLET
                           DLITIAZEM HCL CAPSULE
                             MICRO K 10 CAPSULES
                                         ANTACID
                                         ANTACID
                                 NEXIUM CAPSULES
                              DILTIAZEM CAPSULES
                                      Biestrogen
                      OXYTROL TRANSDERMAL SYSTEM
                                   ALLERGY SHOTS
                      ESTRACE VAGINAL CREAM 0.01
                                         MIRALAX
                                       RANTIDINE
                                 GLUCOSAMINE HCL
                                       synthroid
                                         CELEXIN
                                         CELEXIN
               TIMOLOL GEL FORMING SOLUTION 0.5%
   

In [24]:
#Non Loop Attempt at Spellchecker
from enchant.checker import SpellChecker
chkr = SpellChecker(pwl)
chkr.set_text(pd.Series.to_string(split_meds[1]))
for err in chkr:
    if pwl.suggest(err.word) != []:
        df=(split_meds[1].str.replace(err.word, pwl.suggest(err.word)[0]))
    elif pwl.suggest(err.word) == []:
        df=(split_meds[1].str.replace(err.word, err.word))
    #df=(split_meds[i].str.replace(err.word, pwl.suggest(err.word)[0]))
    #print(df)
    if main_df.empty:
        main_df=pd.Series.to_frame(df)
    else:
        #main_df=pd.concat([main_df, df], axis=1)
        main_df=pd.concat([main_df, df], axis=1)
print(main_df.shape)
print(main_df.head())
             

(1211, 12040)
                                0                               0  \
0                TESTOLIN TABLETS                BYSTOLIC TABLETS   
1                      lisinopril                      LISINOPRIL   
2                        HALFPRIN                        HALFPRIN   
3  TOPROL EXTENDED RELEASE TABLET  TOPROL EXTENDED RELEASE TABLET   
4           DLITIAZEM HCL CAPSULE           DLITIAZEM HCL CAPSULE   

                                 0                              0  \
0                 BYSTOLIC TABLETS               BYSTOLIC TABLETS   
1                       lisinopril                     lisinopril   
2                         HALFPRIN                       HALFPRIN   
3  TOPROL EXTENDRYL RELEASE TABLET  TOPROL EXTENDED RENESE TABLET   
4            DLITIAZEM HCL CAPSULE          DLITIAZEM HCL CAPSULE   

                                0                               0  \
0                BYSTOLIC TABLETS                BYSTOLIC TABLETS   
1                 